In [1]:
import os
import sys
import platform
import pandas as pd

print(sys.version)
print(platform.system())

import re
from datetime import datetime, timedelta

3.7.0 (default, Jun 28 2018, 07:39:16) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Darwin


## Scrap files of the minutes from the BOK website

Scrap files of the minutes from the BOK websit

In [8]:
# create data foler
!mkdir -p ./data/minutes/hwp
!mkdir -p ./data/minutes/txt

In [3]:
!pip install beautifulsoup4

twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
selenium 4.4.0 has requirement urllib3[secure,socks]~=1.26, but you'll have urllib3 1.23 which is incompatible.
pyppeteer 1.0.2 has requirement certifi>=2021, but you'll have certifi 2018.8.24 which is incompatible.
pyppeteer 1.0.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.23 which is incompatible.
Cache entry deserialization failed, entry ignored
You are using pip version 10.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
selenium 4.4.0 has requirement urllib3[secure,socks]~=1.26, but you'll have urllib3 1.23 which is incompatible.
pyppeteer 1.0.2 has requirement certifi>=2021, but you'll have certifi 2018.8.24 which is incompatible.
pyppeteer 1.0.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.23 which is incompatible.
You are using pip vers

In [4]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [9]:
def get_minutes_file(page_addr, mdate, rdate):
  file_header = 'data/minutes/hwp/KO_'
  drive_header = 'drive/My Drive/projects/MPSentiment/data/minutes/hwp/KO_'
  prefix_addr = "http://bok.or.kr"

  page = requests.get(page_addr)
  soup = BeautifulSoup(page.content, 'html.parser')
  links = soup.find('div', class_='addfile').find_all('a')
  for link in links:
      filename = link.get_text()
      filename = filename.replace('\r', '').replace('\t', '').replace('\n', '')
      if filename[-3:] == 'hwp':
          filename = mdate.strftime('%Y%m%d') + "_" + rdate.strftime('%Y%m%d')+ '.hwp'
          file_addr = prefix_addr + link["href"]
          file_res = requests.get(file_addr)

          filepath = file_header + filename
          with open(filepath, 'wb') as f:
              f.write(file_res.content)
          # upload the downloaded file to google drive
          # filepath = drive_header + filename
          # with open(filepath, 'wb') as f:
              # f.write(file_res.content)              
#           file1 = drive.CreateFile({'title': filename})
#           file1.SetContentFile(filepath)
#           file1.Upload()
          
          print(filename, file_addr)

In [6]:
# Because this is for the recent minutes, changed the address of the list to that of the rss feed of the BOK minutes.

def get_minutes_list(from_date='20180101'):
  from_date = datetime.strptime(from_date, '%Y%m%d')
  url = 'https://www.bok.or.kr/portal/bbs/B0000245/news.rss?menuNo=200761'
  user_agent = 'Mozilla/5.0'
  headers ={'User-Agent' : user_agent}
  page = requests.get(url, headers=headers)

  soup = BeautifulSoup(page.content, 'html.parser')
  brdList = soup.find_all('item')

  for post in brdList:
    pubdate = post.find('pubdate').get_text().strip()
    description = post.find('description').get_text().strip()
    guid = post.find('guid').get_text().strip()
    title = post.find('title').get_text().strip()
    if description.replace(' ','').find('통화정책방향') >= 0:
      mdate = title[title.find(')(') + 2:-1]
      if mdate[-1] == '.':
          mdate = mdate[:-1]
      mdate = datetime.strptime(mdate, '%Y.%m.%d')
      if mdate < from_date:
        break
      
      rdate = pubdate[5:16]
      rdate = datetime.strptime(rdate, '%d %b %Y')
#       print(description)
#       print(mdate, rdate, guid)
      get_minutes_file(guid, mdate, rdate)

In [10]:
get_minutes_list()

20190830_20190917.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000013325&fileSn=1
20190718_20190806.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000012634&fileSn=1
20190531_20190618.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000011589&fileSn=1
20190418_20190507.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000010654&fileSn=1
20190228_20190319.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000010656&fileSn=2
20190124_20190212.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000009054&fileSn=1
20181130_20181218.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000007974&fileSn=2
20181018_20181106.hwp http://bok.or.kr/portal/cmmn/file/fileDown.do?menuNo=200761&atchFileId=FILE_000000000007208&fileSn=2
20180831_2018091

## Extact texts from hwp files

In [21]:
!pip install pyhwp

    100% |████████████████████████████████| 225kB 14.1MB/s ta 0:00:01
  Running setup.py bdist_wheel for pyhwp ... done
  Stored in directory: /Users/assa/Library/Caches/pip/wheels/56/d7/7f/050d0b98805dada5b9f8ea498352ff5dc192f2be92422b8ccd
Successfully built pyhwp
twisted 18.7.0 requires PyHamcrest>=1.9.0, which is not installed.
selenium 4.4.0 has requirement urllib3[secure,socks]~=1.26, but you'll have urllib3 1.23 which is incompatible.
pyppeteer 1.0.2 has requirement certifi>=2021, but you'll have certifi 2018.8.24 which is incompatible.
pyppeteer 1.0.2 has requirement urllib3<2.0.0,>=1.25.8, but you'll have urllib3 1.23 which is incompatible.
You are using pip version 10.0.1, however version 23.1.2 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [25]:
!hwp5txt --output ./data/minutes/txt/KO_20190830_20190917.txt ./data/minutes/hwp/KO_20190830_20190917.hwp

In [29]:
filepath = './data/minutes/'
hwp_filelist = os.listdir(filepath + 'hwp')
for hwp_filename in hwp_filelist:
    target_filename = hwp_filename[:-4] + '.txt'
    convert_cmd = f"hwp5txt --output {filepath + 'txt/' + target_filename} {filepath + 'hwp/' + hwp_filename}" 
    os.system(convert_cmd)
    print(convert_cmd)

hwp5txt --output ./data/minutes/txt/KO_20180712_20180731.txt ./data/minutes/hwp/KO_20180712_20180731.hwp
hwp5txt --output ./data/minutes/txt/KO_20190531_20190618.txt ./data/minutes/hwp/KO_20190531_20190618.hwp
hwp5txt --output ./data/minutes/txt/KO_20190830_20190917.txt ./data/minutes/hwp/KO_20190830_20190917.hwp
hwp5txt --output ./data/minutes/txt/KO_20190228_20190319.txt ./data/minutes/hwp/KO_20190228_20190319.hwp
hwp5txt --output ./data/minutes/txt/KO_20181018_20181106.txt ./data/minutes/hwp/KO_20181018_20181106.hwp
hwp5txt --output ./data/minutes/txt/KO_20190718_20190806.txt ./data/minutes/hwp/KO_20190718_20190806.hwp
hwp5txt --output ./data/minutes/txt/KO_20181130_20181218.txt ./data/minutes/hwp/KO_20181130_20181218.hwp
hwp5txt --output ./data/minutes/txt/KO_20190418_20190507.txt ./data/minutes/hwp/KO_20190418_20190507.hwp
hwp5txt --output ./data/minutes/txt/KO_20180227_20180320.txt ./data/minutes/hwp/KO_20180227_20180320.hwp
hwp5txt --output ./data/minutes/txt/KO_20180412_2018050

### Cleasing texts and make a corpus

In [13]:
def tidy_sentences(section):
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'

    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    return sentences, text


In [14]:
def preprocess_minutes(file_path):
    filename = file_path[-24:-4]
    mdate = datetime.strptime(file_path[-21:-13], '%Y%m%d') + timedelta(hours=10)
    rdate = datetime.strptime(file_path[-12:-4], '%Y%m%d') + timedelta(hours=16)

    print('open file: {}'.format(file_path))
    minutes = open(file_path, encoding=u'utf-8').read()

    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여,?|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    # pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,
                    re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    # pos = re.search('((\((다|라)\) )?.?통화정책방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에 관해 다음과 같은 토론이 있었음.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책.?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    # pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes, re.MULTILINE)
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,
                    re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,
                    re.MULTILINE)
    s6 = pos.start() if pos else -1
    # pos = re.search('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    # s7 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)

    print(' ==> text processing completed: {}'.format(filename))

    doc = []

    if any(section_texts):
        for s, (section, sentences) in enumerate(zip(sections, section_texts)):
            for p, text in enumerate(sentences):
                row = (filename, mdate, rdate, section, p, text)
                doc.append(row)
    else:
        print('Empty!')

    return doc

In [15]:
def list_minute_files(path):
    path = os.path.join(path, 'txt')
    for i, file in enumerate(os.listdir(path)):
        file_path = os.path.join(path, file)
        print('--processing {0}th minutes'.format(i+1))
        yield file_path

In [30]:
minutes_path = './data/minutes/'
files = list_minute_files(minutes_path)
docs = []
for file in files:
  doc = preprocess_minutes(file)
  docs += doc

df = pd.DataFrame(docs, columns=['filename', 'mdate', 'rdate', 'section', 'sid', 'text'])
df = df.dropna()
df_minutes_path = os.path.join(minutes_path, 'minutes.csv')
df.to_csv(df_minutes_path, encoding='utf-8', index=False)

--processing 1th minutes
open file: ./data/minutes/txt/KO_20180712_20180731.txt
 ==> text processing completed: KO_20180712_20180731
--processing 2th minutes
open file: ./data/minutes/txt/KO_20190228_20190319.txt
 ==> text processing completed: KO_20190228_20190319
--processing 3th minutes
open file: ./data/minutes/txt/KO_20190718_20190806.txt
 ==> text processing completed: KO_20190718_20190806
--processing 4th minutes
open file: ./data/minutes/txt/KO_20181018_20181106.txt
 ==> text processing completed: KO_20181018_20181106
--processing 5th minutes
open file: ./data/minutes/txt/KO_20190830_20190917.txt
 ==> text processing completed: KO_20190830_20190917
--processing 6th minutes
open file: ./data/minutes/txt/KO_20190531_20190618.txt
 ==> text processing completed: KO_20190531_20190618
--processing 7th minutes
open file: ./data/minutes/txt/KO_20180227_20180320.txt
 ==> text processing completed: KO_20180227_20180320
--processing 8th minutes
open file: ./data/minutes/txt/KO_20180412_20

In [31]:
df

,filename,mdate,rdate,section,sid,text
0,KO_20180712_20180731,2018-07-12 10:00:00,2018-07-31 16:00:00,Economic Situation,0,일부 위원은 세계교역량이 증가하는 등 세계경제가 성장세를 이어가는 상황에서 우리 수...
1,KO_20180712_20180731,2018-07-12 10:00:00,2018-07-31 16:00:00,Economic Situation,1,이어서 동 위원은 최근 서비스의 세계화가 진행되는 가운데 우리 서비스산업의 경쟁력이...
2,KO_20180712_20180731,2018-07-12 10:00:00,2018-07-31 16:00:00,Economic Situation,2,이에 대해 관련부서에서는 일부 서비스산업의 경우 질적 수준이나 생산성이 낮아 서비스...
3,KO_20180712_20180731,2018-07-12 10:00:00,2018-07-31 16:00:00,Economic Situation,3,서비스산업 발전은 오랜 기간 중요한 과제로 제기되어 왔다고 언급하면서 그 방안에 대...
4,KO_20180712_20180731,2018-07-12 10:00:00,2018-07-31 16:00:00,Economic Situation,4,이에 동 위원은 단순하게 서비스 품질을 높이거나 가격을 낮추어야 한다는 측면을 넘어...
...,...,...,...,...,...,...
3635,KO_20180524_20180612,2018-05-24 10:00:00,2018-06-12 16:00:00,Participants’ Views,123,부동산시장 상승세 둔화에도 불구하고 분양수요 등을 고려할 때 주택관련 대출수요가 당...
3636,KO_20180524_20180612,2018-05-24 10:00:00,2018-06-12 16:00:00,Participants’ Views,124,"마지막으로, 미 달러화 강세에 따른 신흥국 금융불안이 커지고 있어 우려됨."
3637,KO_20180524_20180612,2018-05-24 10:00:00,2018-06-12 16:00:00,Participants’ Views,125,"지난 3월 미 연준의 정책금리 인상 후, 크게 흔들리고 있는 일부 국가들뿐만 아니라..."
3638,KO_20180524_20180612,2018-05-24 10:00:00,2018-06-12 16:00:00,Participants’ Views,126,"아직까지 이러한 불안이 신흥국 전체로 확산되지는 않을 것으로 보이고, 우리나라의 경..."


### 금통위 1주일 전 기준으로 news 데이터 분석 수행 및 예측 결과 산출 및 비교

분석 대상 뉴스 데이터 기간 계산

In [41]:
mdate_list = df['mdate'].dt.date.unique()
mdate_list

array([datetime.date(2018, 7, 12), datetime.date(2019, 2, 28),
       datetime.date(2019, 7, 18), datetime.date(2018, 10, 18),
       datetime.date(2019, 8, 30), datetime.date(2019, 5, 31),
       datetime.date(2018, 2, 27), datetime.date(2018, 4, 12),
       datetime.date(2018, 11, 30), datetime.date(2019, 4, 18),
       datetime.date(2019, 1, 24), datetime.date(2018, 8, 31),
       datetime.date(2018, 5, 24)], dtype=object)

In [42]:
mdate_list.sort()

In [43]:
mdate_list

array([datetime.date(2018, 2, 27), datetime.date(2018, 4, 12),
       datetime.date(2018, 5, 24), datetime.date(2018, 7, 12),
       datetime.date(2018, 8, 31), datetime.date(2018, 10, 18),
       datetime.date(2018, 11, 30), datetime.date(2019, 1, 24),
       datetime.date(2019, 2, 28), datetime.date(2019, 4, 18),
       datetime.date(2019, 5, 31), datetime.date(2019, 7, 18),
       datetime.date(2019, 8, 30)], dtype=object)

In [44]:
mdate_list_shift_1w = mdate_list - pd.Timedelta(7, 'd')
mdate_list_shift_1w

array([datetime.date(2018, 2, 20), datetime.date(2018, 4, 5),
       datetime.date(2018, 5, 17), datetime.date(2018, 7, 5),
       datetime.date(2018, 8, 24), datetime.date(2018, 10, 11),
       datetime.date(2018, 11, 23), datetime.date(2019, 1, 17),
       datetime.date(2019, 2, 21), datetime.date(2019, 4, 11),
       datetime.date(2019, 5, 24), datetime.date(2019, 7, 11),
       datetime.date(2019, 8, 23)], dtype=object)

In [45]:
mdate_list[0]

datetime.date(2018, 2, 27)

In [51]:
for i in range(len(mdate_list)):
    print('start_date', 'end_date', 'pred_date')
    if i == 0: 
        print('yyyy-mm-dd', mdate_list_shift_1w[i], mdate_list[i])
    else:
        print(mdate_list_shift_1w[i-1], mdate_list_shift_1w[i], mdate_list[i])

start_date end_date pred_date
yyyy-mm-dd 2018-02-20 2018-02-27
start_date end_date pred_date
2018-02-20 2018-04-05 2018-04-12
start_date end_date pred_date
2018-04-05 2018-05-17 2018-05-24
start_date end_date pred_date
2018-05-17 2018-07-05 2018-07-12
start_date end_date pred_date
2018-07-05 2018-08-24 2018-08-31
start_date end_date pred_date
2018-08-24 2018-10-11 2018-10-18
start_date end_date pred_date
2018-10-11 2018-11-23 2018-11-30
start_date end_date pred_date
2018-11-23 2019-01-17 2019-01-24
start_date end_date pred_date
2019-01-17 2019-02-21 2019-02-28
start_date end_date pred_date
2019-02-21 2019-04-11 2019-04-18
start_date end_date pred_date
2019-04-11 2019-05-24 2019-05-31
start_date end_date pred_date
2019-05-24 2019-07-11 2019-07-18
start_date end_date pred_date
2019-07-11 2019-08-23 2019-08-30


In [54]:
df_mdate = pd.DataFrame([mdate_list, mdate_list_shift_1w]).transpose()
df_mdate.columns = ['mp_date', 'mp_date_1w_ago'] 
df_mdate

,mp_date,mp_date_1w_ago
0,2018-02-27,2018-02-20
1,2018-04-12,2018-04-05
2,2018-05-24,2018-05-17
3,2018-07-12,2018-07-05
4,2018-08-31,2018-08-24
5,2018-10-18,2018-10-11
6,2018-11-30,2018-11-23
7,2019-01-24,2019-01-17
8,2019-02-28,2019-02-21
9,2019-04-18,2019-04-11


In [57]:
df_mdate['mp_date_1w_ago_p1'] = df_mdate['mp_date_1w_ago'].shift(1)
df_mdate

,mp_date,mp_date_1w_ago,mp_date_1w_ago_p1
0,2018-02-27,2018-02-20,NaN
1,2018-04-12,2018-04-05,2018-02-20
2,2018-05-24,2018-05-17,2018-04-05
3,2018-07-12,2018-07-05,2018-05-17
4,2018-08-31,2018-08-24,2018-07-05
5,2018-10-18,2018-10-11,2018-08-24
6,2018-11-30,2018-11-23,2018-10-11
7,2019-01-24,2019-01-17,2018-11-23
8,2019-02-28,2019-02-21,2019-01-17
9,2019-04-18,2019-04-11,2019-02-21


In [58]:
df_mdate[['mp_date_1w_ago_p1', 'mp_date_1w_ago', 'mp_date'] ]

,mp_date_1w_ago_p1,mp_date_1w_ago,mp_date
0,NaN,2018-02-20,2018-02-27
1,2018-02-20,2018-04-05,2018-04-12
2,2018-04-05,2018-05-17,2018-05-24
3,2018-05-17,2018-07-05,2018-07-12
4,2018-07-05,2018-08-24,2018-08-31
5,2018-08-24,2018-10-11,2018-10-18
6,2018-10-11,2018-11-23,2018-11-30
7,2018-11-23,2019-01-17,2019-01-24
8,2019-01-17,2019-02-21,2019-02-28
9,2019-02-21,2019-04-11,2019-04-18
